In [ ]:
!pip install stanza #For google colab

In [ ]:
import pandas as pd
import spacy
import stanza
from stanza.utils.conll import CoNLL
import os
import re
stanza.download('es')
"""def callFreeling(text :str, splitSentences = True) -> list[list[dict]]:
    open(os.path.abspath("")+"/tmp.txt", "w+").write(text)
    os.system(f"bash ./callFreeling.sh {os.path.abspath("")+"/tmp.txt"} {os.path.abspath("")+"/tmp2.txt"}")
    out = open(os.path.abspath("")+"/tmp2.txt", "r").read().split("\n\n") if splitSentences else [open(os.path.abspath("")+"/tmp2.txt", "r").read().replace("\n\n", "\n")]
    os.remove(os.path.abspath("")+"/tmp.txt")
    os.remove(os.path.abspath("")+"/tmp2.txt")
    out = [i.split("\n") for i in out]
    columns = "ID FORM LEMMA TAG SHORT_TAG MSD NEC SENSE SYNTAX DEPHEAD DEPREL COREF SRL".split(" ")
    out = [[{columns[i] : re.split(" +", word)[i] for i in range(len(re.split(" +", word)))} for word in sent]for sent in out]
    return out"""

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: es (Spanish) ...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


'def callFreeling(text :str, splitSentences = True) -> list[list[dict]]:\n    open(os.path.abspath("")+"/tmp.txt", "w+").write(text)\n    os.system(f"bash ./callFreeling.sh {os.path.abspath("")+"/tmp.txt"} {os.path.abspath("")+"/tmp2.txt"}")\n    out = open(os.path.abspath("")+"/tmp2.txt", "r").read().split("\n\n") if splitSentences else [open(os.path.abspath("")+"/tmp2.txt", "r").read().replace("\n\n", "\n")]\n    os.remove(os.path.abspath("")+"/tmp.txt")\n    os.remove(os.path.abspath("")+"/tmp2.txt")\n    out = [i.split("\n") for i in out]\n    columns = "ID FORM LEMMA TAG SHORT_TAG MSD NEC SENSE SYNTAX DEPHEAD DEPREL COREF SRL".split(" ")\n    out = [[{columns[i] : re.split(" +", word)[i] for i in range(len(re.split(" +", word)))} for word in sent]for sent in out]\n    return out'

In [ ]:
#For google collab
!python -m spacy download es_dep_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.8/407.8 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', use_gpu=True)
nlp_spacy = spacy.load("es_dep_news_trf")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
baselineFilePath = "/content/es_ancora-ud-train.conllu" #Baseline corpus path. It's not included in the repo, change this at your own will. Must be in conllu format with UD tags
baseline = CoNLL.conll2doc(baselineFilePath)
sentences = [i.text for i in baseline.sentences]

In [ ]:
# SPACY MODEL
spacy_output = nlp_spacy(" ".join(sentences))

# STANZA MODEL
stanza_output = nlp_stanza(" ".join(sentences))

In [ ]:
# NOTE: Execute this only if you have installed Freeling!!
# FREELING MODEL
freeling_parser = callFreeling(" ".join(sentences))

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/freelingOutput.conll


In [ ]:
# NOTE: Execute this only if you have installed Malt Parser!!
# NOTE: Also check that the Malt Parser folder is in the same directory as the callMaltParser.sh script
# MALT PARSER: chances are this will be removed, as I can't get it to work properly and it depends on freeling sentencizer and tagger
if baselineFilePath[len(baselineFilePath)-6:] != ".conll":
    os.system(f"python conllu-to-conll-standard.py {baselineFilePath} {os.path.abspath("")+"/corpus.conll"}")
    os.system(f"bash callMaltParser.sh {os.path.abspath("")+"/corpus.conll"}")
else:
    os.system(f"bash callMaltParser.sh {baselineFilePath}")
malt_parser = CoNLL.conll2doc(os.path.abspath("")+"/maltParserOut.conll")

/home/ivan/Desktop/Uni/TFG/RepositorioTFG/corpus.conll


-----------------------------------------------------------------------------
                          MaltParser 1.9.2                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Wed Mar 05 17:46:57 CET 2025
  Transition system    : Non-Projective
  Parser configuration : Covington with allow_root=true and allow_shift=false
  Data Format          : /espmalt-1.0/conllx.xml
.          	      1	      2s	    145MB
.          	     10	      2s	    145MB
.          	    100	      3s	    190MB
..........	   1000	     12s	    365MB
.......    	   1721	     15s	    304MB
Parsing time: 00:00:14 (14550 ms)
Finished: Wed Mar 05 17:47

output saved in /home/ivan/Desktop/Uni/TFG/RepositorioTFG/maltParserOut.conll


In [ ]:
print("Baseline: ", len(list(baseline.sentences)), "sentences")
#print("Malt Parser: ", len(list(malt_parser.sentences)), "sentences")
print(f"Spacy: {len(list(spacy_output.sents))} sentences. Baseline difference: ", (len(list(spacy_output.sents))/len(list(baseline.sentences))-1)*100, "%")
print(f"Stanza: {len(list(stanza_output.sentences))} sentences. Baseline difference: ", (len(list(stanza_output.sentences))/len(list(baseline.sentences))-1)*100, "%")
print(f"freeling: {len(freeling_parser)} sentences. Baseline difference: ", (len(freeling_parser)/len(list(baseline.sentences))-1)*100, "%")

Sentence split is inconsistent between stanza, spacy and especially freeling when compared to the baseline, which is a real problem. This highlights the difficulty of comparing syntactic parsers, as there are many more systems that play a role such as PoS tagger and sentencizer. A different approach is required, maybe disabling the sentencizer and processing the corpus sentence by sentence?

## Stanza

In [ ]:
###########################
# SENTENCE SPLIT DISABLED #
###########################
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', tokenize_no_ssplit=True, use_gpu=True)
stanza_output = nlp_stanza("\n\n".join(sentences))

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
tag_dict = {} # Dictionary with a list of length 2 inside. Index 0 for count; index 1 for correct tag in model; index 2 for tag count in model test
for i in range(len(list(stanza_output.sentences))):
    if baseline.sentences[i].text != stanza_output.sentences[i].text:
        print("Sentences are not equal")
    for j in range(len(list(baseline.sentences[i].words))):
        try:
            if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0,0]})
            else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1

            if stanza_output.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({stanza_output.sentences[i].words[j].deprel : [0,0,1]})
            else: tag_dict[stanza_output.sentences[i].words[j].deprel][2] += 1

            if baseline.sentences[i].words[j].deprel == stanza_output.sentences[i].words[j].deprel:
                tag_dict[baseline.sentences[i].words[j].deprel][1] += 1
        except IndexError:
            if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0,0]})
            else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1
            if  j<len(stanza_output.sentences[i].words):
              if stanza_output.sentences[i].words[j].deprel not in tag_dict.keys():
                  tag_dict.update({stanza_output.sentences[i].words[j].deprel : [0,0,1]})
              else: tag_dict[stanza_output.sentences[i].words[j].deprel][2] += 1

In [ ]:
accuracy = {k:tag_dict[k][1]/tag_dict[k][2] if tag_dict[k][2] > 0 else 0 for k in tag_dict.keys()}
print(f"""Precision of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {accuracy["nsubj"]}
\tclause subjects: {accuracy["csubj"]}
\tobjects: {accuracy["obj"]}
\troot of sentence: {accuracy["root"]}
""")

recall = {k:tag_dict[k][1]/tag_dict[k][0] for k in tag_dict.keys()}
print(f"""Recall of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {recall["nsubj"]}
\tclause subjects: {recall["csubj"]}
\tobjects: {recall["obj"]}
\troot of sentence: {recall["root"]}
""")

f1 = {k:(2*recall[k]*accuracy[k])/(recall[k]+accuracy[k]) if (recall[k] > 0 and accuracy[k] > 0) else 0  for k in tag_dict.keys()}
print(f"""F1 of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {f1["nsubj"]}
\tclause subjects: {f1["csubj"]}
\tobjects: {f1["obj"]}
\troot of sentence: {f1["root"]}
""")


Accuracy of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9351116929874805
	clause subjects: 0.8351528384279476
	objects: 0.9107836356865457
	root of sentence: 0.9624133827955484

Recall of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9534854615994326
	clause subjects: 0.7758620689655172
	objects: 0.85728592889334
	root of sentence: 0.9624133827955484

F1 of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9442092000082623
	clause subjects: 0.804416403785489
	objects: 0.883225423685093
	root of sentence: 0.9624133827955484



In [ ]:
##########################
# SENTENCE SPLIT ENABLED #
##########################
tag_dict = {} # Dictionary with a list of length 2 inside. Index 0 for count; index 1 for correct tag in model
neqSents = 0
nlp_stanza = stanza.Pipeline(lang='es', processors='tokenize,ner,mwt,pos,lemma,depparse', use_gpu=True)
for i in range(len(list(baseline.sentences))):
  stanza_output = nlp_stanza(list(baseline.sentences)[i].text)
  if baseline.sentences[i].text != stanza_output.sentences[0].text:
      neqSents+=1
  for j in range(len(list(baseline.sentences[i].words))):
      try:
          if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0,0]})
          else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1

          if stanza_output.sentences[0].words[j].deprel not in tag_dict.keys():
              tag_dict.update({stanza_output.sentences[0].words[j].deprel : [0,0,1]})
          else: tag_dict[stanza_output.sentences[0].words[j].deprel][2] += 1

          if baseline.sentences[i].words[j].deprel == stanza_output.sentences[0].words[j].deprel:
              tag_dict[baseline.sentences[i].words[j].deprel][1] += 1
      except IndexError:
          if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
              tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0]})
          else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1
          if  j<len(stanza_output.sentences[0].words):
            if stanza_output.sentences[0].words[j].deprel not in tag_dict.keys():
                tag_dict.update({stanza_output.sentences[0].words[j].deprel : [0,0,1]})
            else: tag_dict[stanza_output.sentences[0].words[j].deprel][2] += 1

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
accuracy = {k:tag_dict[k][1]/tag_dict[k][2] if tag_dict[k][2] > 0 else 0 for k in tag_dict.keys()}
print(f"""Precision of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {accuracy["nsubj"]}
\tclause subjects: {accuracy["csubj"]}
\tobjects: {accuracy["obj"]}
\troot of sentence: {accuracy["root"]}
""")

recall = {k:tag_dict[k][1]/tag_dict[k][0] for k in tag_dict.keys()}
print(f"""Recall of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {recall["nsubj"]}
\tclause subjects: {recall["csubj"]}
\tobjects: {recall["obj"]}
\troot of sentence: {recall["root"]}
""")

f1 = {k:(2*recall[k]*accuracy[k])/(recall[k]+accuracy[k]) if (recall[k] > 0 and accuracy[k] > 0) else 0  for k in tag_dict.keys()}
print(f"""F1 of Stanza in Spanish subjects, objects and roots:
\tnominal subjects: {f1["nsubj"]}
\tclause subjects: {f1["csubj"]}
\tobjects: {f1["obj"]}
\troot of sentence: {f1["root"]}
""")

print(f"{neqSents} out of {len(sentences)} sentences were not equal, maybe due to a segmentation problem.")

Accuracy of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9354467332757699
	clause subjects: 0.8337004405286343
	objects: 0.9108287056813328
	root of sentence: 0.958843704066634

Recall of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9447305041109543
	clause subjects: 0.7608040201005025
	objects: 0.8510277389742567
	root of sentence: 0.9554331147998326

F1 of Stanza in Spanish subjects, objects and roots:
	nominal subjects: 0.9400656984071235
	clause subjects: 0.7955859169732002
	objects: 0.8799133395233674
	root of sentence: 0.9571353711790392

93 out of 14287 sentences were not equal, maybe due to a segmentation problem.


## Spacy

In [ ]:
# Spacy sentencizer apparently can't be disabled so each sentence in the corpus is processed individually instead. Note that this may risk sentence segmenter interference
spacy.require_gpu()
nlp_spacy = spacy.load("es_dep_news_trf")

In [ ]:
tag_dict = {} # Dictionary with a list of length 2 inside. Index 0 for count; index 1 for correct tag in model
neqSents = 0
for i in range(len(baseline.sentences)):
    spacy_output = nlp_spacy(baseline.sentences[i].text.replace(" del ", " de el "))
    if baseline.sentences[i].text != list(spacy_output.sents)[0].text:
         neqSents+=1
    for j in range(len(baseline.sentences[i].words)):
        if baseline.sentences[i].words[j].deprel not in tag_dict.keys():
                tag_dict.update({baseline.sentences[i].words[j].deprel : [1,0,0]})
        else: tag_dict[baseline.sentences[i].words[j].deprel][0] += 1
        if j < len(list(spacy_output.sents)[0]):
            if list(spacy_output.sents)[0][j].dep_.lower() not in tag_dict.keys():
                tag_dict.update({list(spacy_output.sents)[0][j].dep_.lower(): [0,0,1]})
            else: tag_dict[list(spacy_output.sents)[0][j].dep_.lower()] [2]+= 1
            """print(f"{baseline.sentences[i].words[j].text} / {list(spacy_output.sents)[0][j+offset].text} with deprel {baseline.sentences[i].words[j].deprel} / {list(spacy_output.sents)[0][j+offset].dep_.lower()}")""" # This is for debugging purposes
            if baseline.sentences[i].words[j].deprel == list(spacy_output.sents)[0][j].dep_.lower():
                tag_dict[baseline.sentences[i].words[j].deprel][1] += 1


In [ ]:
accuracy = {k:tag_dict[k][1]/tag_dict[k][2] if tag_dict[k][2] > 0 else 0 for k in tag_dict.keys()}
print(f"""Precision of Spacy in Spanish subjects, objects and roots:
\tnominal subjects: {accuracy["nsubj"]}
\tclause subjects: {accuracy["csubj"]}
\tobjects: {accuracy["obj"]}
\troot of sentence: {accuracy["root"]}
""")

recall = {k:tag_dict[k][1]/tag_dict[k][0]  if (tag_dict[k][0] > 0 and tag_dict[k][1] > 0) else 0  for k in tag_dict.keys()}
print(f"""Recall of Spacy in Spanish subjects, objects and roots:
\tnominal subjects: {recall["nsubj"]}
\tclause subjects: {recall["csubj"]}
\tobjects: {recall["obj"]}
\troot of sentence: {recall["root"]}
""")

f1 = {k:(2*recall[k]*accuracy[k])/(recall[k]+accuracy[k]) if (recall[k] > 0 and accuracy[k] > 0) else 0  for k in tag_dict.keys()}
print(f"""F1 of Spacy in Spanish subjects, objects and roots:
\tnominal subjects: {f1["nsubj"]}
\tclause subjects: {f1["csubj"]}
\tobjects: {f1["obj"]}
\troot of sentence: {f1["root"]}
""")
print(f"{neqSents} out of {len(sentences)} sentences were not equal, maybe due to a segmentation problem.")

Accuracy of Spacy in Spanish subjects, objects and roots:
	nominal subjects: 0.893353941267388
	clause subjects: 0.8672032193158954
	objects: 0.626152256834075
	root of sentence: 0.9418311633767324

Recall of Spacy in Spanish subjects, objects and roots:
	nominal subjects: 0.8921613616453214
	clause subjects: 0.8742393509127789
	objects: 0.7892917960532906
	root of sentence: 0.9417652411282984

F1 of Spacy in Spanish subjects, objects and roots:
	nominal subjects: 0.8927572531830517
	clause subjects: 0.8707070707070708
	objects: 0.6983205565648957
	root of sentence: 0.9417982010989395

4424 out of 14287 sentences were not equal, maybe due to a segmentation problem.
